In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from collections import Counter


In [ ]:
## resize the image
from skimage import io
import matplotlib.pyplot as plt
from skimage.transform import resize
%matplotlib inline

def resize_image(image_url):
    image = io.imread(image_url)
    image = resize(image, (448,448))
    return image
  
  

In [ ]:
# create json file for vocabulary
import json
import os
import nltk
from nltk.stem.snowball import *
from tqdm import *
from collections import Counter, OrderedDict
import string


annotation_dir = "../input/vizwizvarun/Annotations/Annotations/"


## question
q_counter = Counter()
n_sample = 0
maxlen = 0
train_data = json.load(open(annotation_dir + 'train.json'))
for one_data in tqdm(train_data):
    n_sample += 1
    question = one_data['question']
    question = question.lower()
    tokens = nltk.word_tokenize(question)
    token_len = len(tokens)
    maxlen = max([maxlen,token_len])
    q_counter.update(tokens)
print('number of sample = ' + str(n_sample))
print('max len = ' + str(maxlen))
q_word_counts = [x for x in q_counter.items()]
q_word_counts.sort(key=lambda x: x[1], reverse=True)
json.dump(q_word_counts, open('q_word_counts.json', "w"), indent=2)

### build vocabulary based on question
vocab = [x[0] for x in q_word_counts if x[1] >= 0]
unk_word = '<UNK>'
vocab = [unk_word] + vocab
vocab = OrderedDict(zip(vocab,range(len(vocab))))
json.dump(vocab, open('word2vocab_id.json', 'w'), indent=2)

## answer
ans_counter = Counter()

train_data = json.load(open(annotation_dir + 'train.json'))
for annotation in tqdm(train_data):
    for answer in annotation['answers']:
        answer = answer['answer'].lower()
        ans_counter.update([answer]) # don't forget the [], counter.update input a list
ans_counts = [x for x in ans_counter.items()]
ans_counts.sort(key=lambda x: x[1], reverse=True)
json.dump(ans_counts, open('ans_counts.json', "w"), indent=2)

### build answer candidates
output_num = 3000
n_totoal = sum([x[1] for x in ans_counts])
ans_counts = ans_counts[:output_num]
n_cover = sum([x[1] for x in ans_counts])
print("we keep top %d most answers"%len(ans_counts))
print("coverage: %d/%d (%.4f)"%(n_cover, n_totoal, 1.0 * n_cover / n_totoal))
ans_list = [x[0] for x in ans_counts]
ans_list.append('<unk>')
ans_dict = OrderedDict(zip(ans_list,range(len(ans_list))))
json.dump(ans_dict, open('answer2answer_id.json', 'w'), indent=2)

In [ ]:
qvocabsize=0
data=json.load(open("./q_word_counts.json"))
for i in data:
    qvocabsize+=1

In [ ]:
qvocabsize

In [ ]:
#Resize the images to 448*448 and save the training images
from PIL import Image
from tqdm import *
import numpy as np
from skimage.io import imsave, imread
import os
import json
import sys


  
image_size = 448


main_dir = "../input/vizwizvarun"
train_dataset = main_dir + '/train/train'
output="images_trainedresized"

images_name = os.listdir(train_dataset)
images_name.sort()
print(len(images_name))
if not os.path.exists(output):
    os.makedirs(output) 
for idx, img_info in enumerate(tqdm(images_name)):
    
    img_path = train_dataset+"/"+img_info
    
    image = resize_image(img_path)
    imsave(os.path.join(output, img_info),image)
    
    

In [ ]:
#Resize the images to 448*448 and save the validation images
from tqdm import *
import numpy as np
import os
import json
from PIL import Image
import sys
from skimage.io import imsave, imread

  
image_size = 448

main_dir = "../input/vizwizvarun"
output="images_resized_val"
dataset_dir = main_dir + '/val/val'
images_name = os.listdir(dataset_dir)
images_name.sort()
print(len(images_name))
if not os.path.exists(output):
    os.makedirs(output) 
for idx, img_info in enumerate(tqdm(images_name)):
    img_path = dataset_dir+"/"+img_info
    image = resize_image(img_path)
    imsave(os.path.join(output, img_info),image)

In [ ]:
from tqdm import *
import numpy as np
import os
import json
from PIL import Image




import sys
from skimage.io import imsave, imread

  
image_size = 448

main_dir = "../input/vizwizvarun"



output="images_testresized"
dataset_dir = main_dir + '/test/test'
all_imgs = os.listdir(dataset_dir)
all_imgs.sort()
print(len(all_imgs))
if not os.path.exists(output):
    os.makedirs(output) 
for idx, img_info in enumerate(tqdm(all_imgs[0:1000])):
    
    img_path = dataset_dir+"/"+img_info
    
    image = resize_image(img_path)
    imsave(os.path.join(output, img_info),image)
    

In [ ]:
import json
import numpy as np
import nltk
import os
from tqdm import *
from collections import Counter
import re
output_dir="data"

def convert_to_tokens(sentence):

    regex = re.compile(r'(\W+)')
    tokens = regex.split(sentence.lower())
    tokens = [w.strip() for w in tokens if len(w.strip()) > 0]
    return tokens

root_dir = "../input/vizwizvarun/Annotations/Annotations"
#splits = ['train', 'val', 'test']

dirs="../input/vizwizvarun/"
vocab = json.load(open('./word2vocab_id.json'))
answer2answer_id = json.load(open('./answer2answer_id.json'))
#../input/vizwizvarun/train/train/VizWiz_train_00000000.jpg



In [ ]:
def sentence_to_vocabid(sentence, vocab):
    unk_word = '<UNK>'
    tokens = nltk.word_tokenize(sentence.lower())
    tokens_id = [vocab.get(x, vocab[unk_word]) for x in tokens]
    return tokens_id

In [ ]:
def preprocessingtrain(type1):
        missing_images = []

   
    ## load annotation file and question file
        dataset = json.load(open(root_dir+'/'+type1+'.json'))
        data = [None]*len(dataset)

        for idx,one_data in enumerate(dataset):
            ans_counter = Counter([x['answer'] for x in one_data['answers']])
            allanswers = [x['answer'] for x in one_data['answers']]
            ans = ans_counter.most_common(1)[0][0]
            a_label = answer2answer_id.get(ans,3000)
            answers=[]
#             if type1 == 'train' and a_label == -1:
#                 continue
            for answer in allanswers :
                answers.append(answer2answer_id.get(answer,3000))
            
       
            imagename = one_data['image']
            imagepath = "../input/trainresized/images_trainedresized" "/"+ imagename
            questionsentence=one_data['question']
            questiontokens = nltk.word_tokenize(question)
            questionencoding = sentence_to_vocabid(one_data['question'], vocab)
            if(len(questionencoding)>=37):
                questionencoding=questionencoding[0:37]
            else:
                questionencoding.extend([0]*(37-len(questionencoding)))
            image_info = {'imagename': imagename,
                'imagepath': imagepath,
                'questionsentence': questionsentence,
                'questiontokens': questiontokens
                }
            
            image_info['questionencoding'] = questionencoding
            image_info['selectedanswer'] = ans
            image_info['answerlabel']=a_label
            image_info['answers']=answers
            data[idx]=image_info
        return data

In [ ]:
def preprocessingvalidation(type1):
        missing_images = []

   
    ## load annotation file and question file
        dataset = json.load(open(root_dir+'/'+type1+'.json'))
        data = [None]*len(dataset)

        for idx,one_data in enumerate(dataset):
            ans_counter = Counter([x['answer'] for x in one_data['answers']])
            allanswers = [x['answer'] for x in one_data['answers']]
            ans = ans_counter.most_common(1)[0][0]
            a_label = answer2answer_id.get(ans,3000)
            answers=[]
#             if type1 == 'val' and a_label == -1:
#                 continue
            for answer in allanswers :
                answers.append(answer2answer_id.get(answer,3000))
            
       
            imagename = one_data['image']
            imagepath = "../input/validationresized/images_resized_val" "/"+ imagename
            questionsentence=one_data['question']
            questiontokens = nltk.word_tokenize(question)
            questionencoding = sentence_to_vocabid(one_data['question'], vocab)
            if(len(questionencoding)>=37):
                questionencoding=questionencoding[0:37]
            else:
                questionencoding.extend([0]*(37-len(questionencoding)))
            
            image_info = {'imagename': imagename,
                'imagepath': imagepath,
                'questionsentence': questionsentence,
                'questiontokens': questiontokens
                         }
            
            image_info['questionencoding'] = questionencoding
            image_info['selectedanswer'] = ans
            image_info['answerlabel']=a_label
            image_info['answers']=answers
            data[idx]=image_info
        return data

In [ ]:
def sampledata(data,num):
    import random
    random.shuffle(data)
    return data[0:num]

In [ ]:
preprecessed={}
preprecessed['train']=sampledata(preprocessingtrain('train'),10000)
preprecessed['val']=sampledata(preprocessingvalidation('val'),1000)
preprecessed['train-val'] = preprecessed['train'] + preprecessed['val']


In [ ]:
len(preprecessed['val'])

In [ ]:
os.makedirs("data")

In [ ]:
 for key, value in preprecessed.items():
        np.save(os.path.join("./data", f'{key}.npy'), np.array(value))

In [ ]:
import numpy as np
import os

from torch.utils.data import Dataset
from torchvision import transforms
from torch.utils.data import DataLoader
from PIL import Image

INPUT_DIR = './data'

class VizWizdataset(Dataset):

    def __init__(self, input_dir, file, transformimage = None):

        self.input_data = np.load(os.path.join(input_dir, file), allow_pickle=True)
        self.nottest = True if not "test" in file else False
        self.transformimage = transformimage

    def __getitem__(self, idx):
        if(self.input_data[idx] is None): 
            path = self.input_data[1]['imagepath']
            img = np.array(Image.open(path))
            questiontokens = self.input_data[1]['questiontokens']
            questiontoid=self.input_data[1]['questionencoding']
            if(len(questiontoid)>=37):
                questiontoid=questiontoid[0:37]
            else:
                questiontoid=questiontoid.extend([0]*(37-len(questiontoid)))
            questiontoid=(np.asarray(self.input_data[1]['questionencoding']))
            
            dataentry = {'image': img, 'question': questiontoid}
            if(self.nottest):
                dataentry['answers']=np.array(self.input_data[1]['answers'])
            if self.transformimage:
                dataentry['image'] = self.transformimage(dataentry['image'])
                dataentry['imagepath']=self.input_data[1]['imagepath']
            if self.nottest:
                answertoid = self.input_data[1]['answerlabel']
                dataentry['answer'] = answertoid
            return dataentry
        else:
            path = self.input_data[idx]['imagepath']
            #print(path)
            img = np.array(Image.open(path))
            questiontokens = self.input_data[idx]['questiontokens']
            questiontoid=self.input_data[idx]['questionencoding']
            if(len(questiontoid)>=37):
                questiontoid=questiontoid[0:37]
            else:
                questiontoid=questiontoid.extend([0]*(37-len(questiontoid)))
            questiontoid=(np.asarray(self.input_data[idx]['questionencoding']))
            dataentry = {'image': img, 'question': questiontoid}
            if(self.nottest):
                dataentry['answers']=np.array(self.input_data[idx]['answers'])
            if self.nottest:
                answertoid = self.input_data[idx]['answerlabel']
                dataentry['answer'] = answertoid

            if self.transformimage:
                dataentry['image'] = self.transformimage(dataentry['image'])
                dataentry['imagepath']=self.input_data[idx]['imagepath']

            return dataentry

    def __len__(self):

        return len(self.input_data)

def data_loader(input_dir, batch_size, num_worker):

    transformimage = transforms.Compose([
        transforms.ToTensor(),  # convert to a tensor
        transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    )                          # normalize the image so pizel values lie between 0 and 1
    ])

    vizwiz = {
        'train': VizWizdataset(
            input_dir=input_dir,
            file='train.npy',
            transformimage=transformimage),
        'val': VizWizdataset(
            input_dir=input_dir,
            file='val.npy',
            transformimage=transformimage)
    }

    dataloader = {
        'train': DataLoader(
            dataset=vizwiz['train'],
            batch_size=batch_size,
            shuffle=True,
            num_workers=num_worker
            ),
        'val': DataLoader(
            dataset=vizwiz['val'],
            batch_size=batch_size,
            shuffle=True,
            num_workers=num_worker
            )
        
    }

    return dataloader



In [ ]:
#Model
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F


class ImgPreprocess(nn.Module):

    def __init__(self, embed_dim):

        super(ImgPreprocess, self).__init__()
        self.model = models.resnet50(pretrained=True)
        in_features = self.model.fc.in_features
        self.model.fc =  nn.Linear(in_features, embed_dim)# remove resnet50 last layer
      

    def forward(self, image):
        with torch.no_grad():
            imagefeatures = self.model(image) # (batch, channel, height, width)
            
        l2_norm = F.normalize(imagefeatures, p=2, dim=1).detach()
        return l2_norm

class QuestionPreprocess(nn.Module):
    def __init__(self, questionvocabularysize, wordembeddingsize, hidden_size, num_hidden, sizeofconcatenatedlayer):
                super(QuestionPreprocess, self).__init__()
                self.word_embedding = nn.Embedding(questionvocabularysize, wordembeddingsize)
                self.tanh = nn.Tanh()
                self.hidden_size=hidden_size
                self.num_hidden=num_hidden
                self.lstm=nn.LSTM(wordembeddingsize,hidden_size,num_hidden,batch_first=True,bidirectional=True)
                self.fc=nn.Linear(hidden_size*2,sizeofconcatenatedlayer)
    def forward(self, question):
                qu_embedding = self.word_embedding(question)  
                qu_embedding = self.tanh(qu_embedding)
                h0=torch.zeros(self.num_hidden*2,qu_embedding.size(0),self.hidden_size).to('cuda')
                c0=torch.zeros(self.num_hidden*2,qu_embedding.size(0),self.hidden_size).to('cuda')
                output,_=self.lstm(qu_embedding,(h0,c0))
                output=self.fc(output[:,-1,:])
                return output
class VizWizModel(nn.Module):

    def __init__(self, sizeoffeatures, questionvocabularysize, answervocabularysize, wordembeddingsize, hidden_size, num_hidden):

        super(VizWizModel, self).__init__()
        self.questionpreprocess = QuestionPreprocess(questionvocabularysize, wordembeddingsize, hidden_size, num_hidden, sizeoffeatures)
        self.imgpreprocess = ImgPreprocess(sizeoffeatures)
        self.tanh = nn.Tanh()
        self.dropout = nn.Dropout(0.5)
        self.layer1 = nn.Linear(sizeoffeatures, answervocabularysize)
        self.layer2 = nn.Linear(answervocabularysize, answervocabularysize)

    def forward(self, image, question):

        imagefeatures = self.imgpreprocess(image)               
        queationfeatures = self.questionpreprocess(question)
        dotproductfeatures = imagefeatures * queationfeatures
        dotproductfeatures = self.dropout(dotproductfeatures)
        dotproductfeatures = self.tanh(dotproductfeatures)
        dotproductfeatures = self.layer1(dotproductfeatures)       
        dotproductfeatures = self.dropout(dotproductfeatures)
        dotproductfeatures = self.tanh(dotproductfeatures)
        logits = self.layer2(dotproductfeatures)                 

        return logits


In [ ]:
import os
import time
import torch
import torch.nn as nn

from torch import optim


train_accuracy=[]
valid_accuracy=[]
epochs_total=[]
datadir = './data'
model_dir = './ckpt'
logs = './log'

workers = 1
batchsize = 100
sizeoffeatures, embeddingsize = 1024, 300
numofhidden, sizeofhiddenlayer = 3, 512

epochs = 10

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device='cuda'
print(device)
def train():
    dataloader = data_loader(input_dir=datadir, batch_size=batchsize, num_worker=workers)
    questionvocabularysize = qvocabsize+1
    answervocabularysize = 3001
    
    model = VizWizModel(sizeoffeatures=sizeoffeatures, questionvocabularysize=questionvocabularysize, answervocabularysize=answervocabularysize,wordembeddingsize=embeddingsize, hidden_size=sizeofhiddenlayer, num_hidden=numofhidden).to(device)
    
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
   
   
    print('>> start training')
    start_time = time.time()
    for epoch in range(epochs):
        metrics=[]
        metrics1=[]
        correct1=0
        length1=0
        correct=0
        length=0
        epoch_loss = {key: 0 for key in ['train', 'val']}
        print(epoch)
        model.train()
        print(dataloader['train'])
        for idx, dataentry in enumerate(dataloader['train']):
            
            image = dataentry['image'].to(device)
            question = dataentry['question'].to(device)
            label = dataentry['answer'].to(device)
            answers=dataentry['answers']
            label1=label.tolist()
            
            # forwardpropogation
            logits = model(image, question)
            
            with torch.no_grad():
                predict = torch.log_softmax(logits, dim=1)
                predict = torch.argmax(predict, dim=1).tolist()
            print("Train set prediction {}".format(Counter(predict)))
            for i in range(len(label1)):
                if(predict[i]==label1[i]):
                    correct+=1
            length=length+batchsize
            for i in range(answers.shape[0]):
                count=0
                for j in range(answers.shape[1]):
                    if(answers[i][j]==predict[i]):
                        count+=1
                metrics.append(min(count//3,1))
            loss = criterion(logits, label)
            epoch_loss['train'] += loss.item()
            # backpropogation
            optimizer.zero_grad()
            loss.backward()
            print(loss.item())
            optimizer.step()
        model.eval()
        correct = correct/length 
        print("Train set Accuracy is {}".format(correct))
        print("Train set Accuracy metric is {}".format(np.mean(metrics)))
        train_accuracy.append(np.mean(metrics))
        
        for idx, dataentry in enumerate(dataloader['val']):
            image = dataentry['image'].to(device=device)
            question = dataentry['question'].to(device=device)
            label = dataentry['answer'].to(device=device)
            answers=dataentry['answers']
            label1=label
            with torch.no_grad():
                logits = model(image, question)
                loss = criterion(logits, label)
            epoch_loss['val'] += loss.item()
            with torch.no_grad():
                predict = torch.log_softmax(logits, dim=1)
                predict = torch.argmax(predict, dim=1).tolist()
            print("Valid set prediction {}".format(Counter(predict)))
            for i in range(len(predict)):
                if(predict[i]==label1[i]):
                    correct1+=1
            length1=length1+batchsize
            for i in range(answers.shape[0]):
                count=0
                for j in range(answers.shape[1]):
                    if(answers[i][j]==predict[i]):
                        count+=1
                metrics1.append(min(count//3,1))
        correct1 = correct1/length1
        print("Validation set Accuracy is {}".format(correct1))
        print("Validation set Accuracy metric is {}".format(np.mean(metrics1)))
        valid_accuracy.append(np.mean(metrics1))
        epochs_total.append(epoch)
        # statistic
        for phase in ['train', 'val']:
            epoch_loss[phase] /= len(dataloader[phase])
            with open(os.path.join(logs, f'{phase}_log.txt'), 'a') as f:
                f.write(str(epoch+1) + '\t' + str(epoch_loss[phase]) + '\n')
        print('Epoch:{}/{} | Training Loss: {train:6f} | Validation Loss: {val:6f}'.format(epoch+1, epochs, **epoch_loss))
          
        torch.save(model.state_dict(), os.path.join(model_dir, f'model-epoch-{epoch+1}.pth'))

        scheduler.step()
    end_time = time.time()
    training_time = end_time - start_time
    print("Total training time in mins is {}".format(training_time//60))
    
if not os.path.exists(logs):
    os.makedirs(logs)
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
train()

In [ ]:

# importing package
import matplotlib.pyplot as plt
import numpy as np

# plot lines
plt.plot(epochs_total, train_accuracy, label = "train_accuracy")
plt.plot(epochs_total, valid_accuracy, label = "validation_accuracy")
plt.legend()
plt.show()

In [ ]:
import os
import time
import torch
import torch.nn as nn

from torch import optim


train_accuracy=[]
valid_accuracy=[]
epochs_total=[]
datadir = './data'
model_dir = './ckpt'
logs = './log'

workers = 1
batchsize = 100
sizeoffeatures, embeddingsize = 1024, 1
sizeoffeatures, embeddingsize = 1024, 150
numofhidden, sizeofhiddenlayer = 2, 256

epochs = 10

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device='cuda'
print(device)
def train():
    dataloader = data_loader(input_dir=datadir, batch_size=batchsize, num_worker=workers)
    questionvocabularysize = qvocabsize+1
    answervocabularysize = 3001
    
    model = VizWizModel(sizeoffeatures=sizeoffeatures, questionvocabularysize=questionvocabularysize, answervocabularysize=answervocabularysize,wordembeddingsize=embeddingsize, hidden_size=sizeofhiddenlayer, num_hidden=numofhidden).to(device)
    
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
   
   
    print('>> start training')
    start_time = time.time()
    for epoch in range(epochs):
        metrics=[]
        metrics1=[]
        correct1=0
        length1=0
        correct=0
        length=0
        epoch_loss = {key: 0 for key in ['train', 'val']}
        print(epoch)
        model.train()
        print(dataloader['train'])
        for idx, dataentry in enumerate(dataloader['train']):
            
            image = dataentry['image'].to(device)
            question = dataentry['question'].to(device)
            label = dataentry['answer'].to(device)
            answers=dataentry['answers']
            label1=label.tolist()
            
            # forwardpropogation
            logits = model(image, question)
            
            with torch.no_grad():
                predict = torch.log_softmax(logits, dim=1)
                predict = torch.argmax(predict, dim=1).tolist()
            print("Train set prediction {}".format(Counter(predict)))
            for i in range(len(label1)):
                if(predict[i]==label1[i]):
                    correct+=1
            length=length+batchsize
            for i in range(answers.shape[0]):
                count=0
                for j in range(answers.shape[1]):
                    if(answers[i][j]==predict[i]):
                        count+=1
                metrics.append(min(count//3,1))
            loss = criterion(logits, label)
            epoch_loss['train'] += loss.item()
            # backpropogation
            optimizer.zero_grad()
            loss.backward()
            print(loss.item())
            optimizer.step()
        model.eval()
        correct = correct/length 
        print("Train set Accuracy is {}".format(correct))
        print("Train set Accuracy metric is {}".format(np.mean(metrics)))
        train_accuracy.append(np.mean(metrics))
        
        for idx, dataentry in enumerate(dataloader['val']):
            image = dataentry['image'].to(device=device)
            question = dataentry['question'].to(device=device)
            label = dataentry['answer'].to(device=device)
            answers=dataentry['answers']
            label1=label
            with torch.no_grad():
                logits = model(image, question)
                loss = criterion(logits, label)
            epoch_loss['val'] += loss.item()
            with torch.no_grad():
                predict = torch.log_softmax(logits, dim=1)
                predict = torch.argmax(predict, dim=1).tolist()
            print("Valid set prediction {}".format(Counter(predict)))
            for i in range(len(predict)):
                if(predict[i]==label1[i]):
                    correct1+=1
            length1=length1+batchsize
            for i in range(answers.shape[0]):
                count=0
                for j in range(answers.shape[1]):
                    if(answers[i][j]==predict[i]):
                        count+=1
                metrics1.append(min(count//3,1))
        correct1 = correct1/length1
        print("Validation set Accuracy is {}".format(correct1))
        print("Validation set Accuracy metric is {}".format(np.mean(metrics1)))
        valid_accuracy.append(np.mean(metrics1))
        epochs_total.append(epoch)
        # statistic
        for phase in ['train', 'val']:
            epoch_loss[phase] /= len(dataloader[phase])
            with open(os.path.join(logs, f'{phase}_log.txt'), 'a') as f:
                f.write(str(epoch+1) + '\t' + str(epoch_loss[phase]) + '\n')
        print('Epoch:{}/{} | Training Loss: {train:6f} | Validation Loss: {val:6f}'.format(epoch+1, epochs, **epoch_loss))
          
        torch.save(model.state_dict(), os.path.join(model_dir, f'model-epoch-{epoch+1}.pth'))

        scheduler.step()
    end_time = time.time()
    training_time = end_time - start_time
    print("Total training time in mins is {}".format(training_time//60))
    
if not os.path.exists(logs):
    os.makedirs(logs)
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
train()

In [ ]:

# importing package
import matplotlib.pyplot as plt
import numpy as np

# plot lines
plt.plot(epochs_total, train_accuracy, label = "train_accuracy")
plt.plot(epochs_total, valid_accuracy, label = "validation_accuracy")
plt.legend()
plt.show()

In [ ]:
#Model
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F


class ImgPreprocess(nn.Module):

    def __init__(self, embed_dim):

        super(ImgPreprocess, self).__init__()
        self.model = models.vgg19(pretrained=True)
        in_features = self.model.classifier[-1].in_features
        self.model.classifier[-1] =  nn.Linear(in_features, embed_dim)# remove resnet50 last layer
      
    def forward(self, image):

        with torch.no_grad():
           
            imagefeatures = self.model(image) # (batch, channel, height, width)
        l2_norm = F.normalize(imagefeatures, p=2, dim=1).detach()
        return l2_norm

class QuestionPreprocess(nn.Module):

    def __init__(self, questionvocabularysize, wordembeddingsize, hidden_size, num_hidden, sizeofconcatenatedlayer):
                super(QuestionPreprocess, self).__init__()
                self.word_embedding = nn.Embedding(questionvocabularysize, wordembeddingsize)
                self.tanh = nn.Tanh()
                self.hidden_size=hidden_size
                self.num_hidden=num_hidden
                self.lstm=nn.LSTM(wordembeddingsize,hidden_size,num_hidden,batch_first=True,bidirectional=True)
                self.fc=nn.Linear(hidden_size*2,sizeofconcatenatedlayer)
    def forward(self, question):
                qu_embedding = self.word_embedding(question)  
                qu_embedding = self.tanh(qu_embedding)
                output,_=self.lstm(qu_embedding)
                output=self.fc(output[:,-1,:])
                return output
class VizWizModel(nn.Module):

    def __init__(self, sizeoffeatures, questionvocabularysize, answervocabularysize, wordembeddingsize, hidden_size, num_hidden):

        super(VizWizModel, self).__init__()
        self.questionpreprocess = QuestionPreprocess(questionvocabularysize, wordembeddingsize, hidden_size, num_hidden, sizeoffeatures)
        self.imgpreprocess = ImgPreprocess(sizeoffeatures)
        self.tanh = nn.Tanh()
        self.dropout = nn.Dropout(0.5)
        self.layer1 = nn.Linear(sizeoffeatures, answervocabularysize)
        self.layer2 = nn.Linear(answervocabularysize, answervocabularysize)

    def forward(self, image, question):

        imagefeatures = self.imgpreprocess(image)               
        queationfeatures = self.questionpreprocess(question)
        dotproductfeatures = imagefeatures * queationfeatures
        dotproductfeatures = self.dropout(dotproductfeatures)
        dotproductfeatures = self.tanh(dotproductfeatures)
        dotproductfeatures = self.layer1(dotproductfeatures)       
        dotproductfeatures = self.dropout(dotproductfeatures)
        dotproductfeatures = self.tanh(dotproductfeatures)
        logits = self.layer2(dotproductfeatures)                 

        return logits


In [ ]:
import os
import time
import torch
import torch.nn as nn

from torch import optim


train_accuracy=[]
valid_accuracy=[]
epochs_total=[]
datadir = './data'
model_dir = './ckpt'
logs = './log'

workers = 1
batchsize = 100
sizeoffeatures, embeddingsize = 1024, 300
numofhidden, sizeofhiddenlayer = 3, 512

epochs = 10

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device='cuda'
print(device)
def train():
    dataloader = data_loader(input_dir=datadir, batch_size=batchsize, num_worker=workers)
    questionvocabularysize = qvocabsize+1
    answervocabularysize = 3001
    
    model = VizWizModel(sizeoffeatures=sizeoffeatures, questionvocabularysize=questionvocabularysize, answervocabularysize=answervocabularysize,wordembeddingsize=embeddingsize, hidden_size=sizeofhiddenlayer, num_hidden=numofhidden).to(device)
    
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
   
   
    print('>> start training')
    start_time = time.time()
    for epoch in range(epochs):
        metrics=[]
        metrics1=[]
        correct1=0
        length1=0
        correct=0
        length=0
        epoch_loss = {key: 0 for key in ['train', 'val']}
        print(epoch)
        model.train()
        print(dataloader['train'])
        for idx, dataentry in enumerate(dataloader['train']):
            
            image = dataentry['image'].to(device)
            question = dataentry['question'].to(device)
            label = dataentry['answer'].to(device)
            answers=dataentry['answers']
            label1=label.tolist()
            
            # forwardpropogation
            logits = model(image, question)
            
            with torch.no_grad():
                predict = torch.log_softmax(logits, dim=1)
                predict = torch.argmax(predict, dim=1).tolist()
            print("Train set prediction {}".format(Counter(predict)))
            for i in range(len(label1)):
                if(predict[i]==label1[i]):
                    correct+=1
            length=length+batchsize
            for i in range(answers.shape[0]):
                count=0
                for j in range(answers.shape[1]):
                    if(answers[i][j]==predict[i]):
                        count+=1
                metrics.append(min(count//3,1))
            loss = criterion(logits, label)
            epoch_loss['train'] += loss.item()
            # backpropogation
            optimizer.zero_grad()
            loss.backward()
            print(loss.item())
            optimizer.step()
        model.eval()
        correct = correct/length 
        print("Train set Accuracy is {}".format(correct))
        print("Train set Accuracy metric is {}".format(np.mean(metrics)))
        train_accuracy.append(np.mean(metrics))
        
        for idx, dataentry in enumerate(dataloader['val']):
            image = dataentry['image'].to(device=device)
            question = dataentry['question'].to(device=device)
            label = dataentry['answer'].to(device=device)
            answers=dataentry['answers']
            label1=label
            with torch.no_grad():
                logits = model(image, question)
                loss = criterion(logits, label)
            epoch_loss['val'] += loss.item()
            with torch.no_grad():
                predict = torch.log_softmax(logits, dim=1)
                predict = torch.argmax(predict, dim=1).tolist()
            print("Valid set prediction {}".format(Counter(predict)))
            for i in range(len(predict)):
                if(predict[i]==label1[i]):
                    correct1+=1
            length1=length1+batchsize
            for i in range(answers.shape[0]):
                count=0
                for j in range(answers.shape[1]):
                    if(answers[i][j]==predict[i]):
                        count+=1
                metrics1.append(min(count//3,1))
        correct1 = correct1/length1
        print("Validation set Accuracy is {}".format(correct1))
        print("Validation set Accuracy metric is {}".format(np.mean(metrics1)))
        valid_accuracy.append(np.mean(metrics1))
        epochs_total.append(epoch)
        # statistic
        for phase in ['train', 'val']:
            epoch_loss[phase] /= len(dataloader[phase])
            with open(os.path.join(logs, f'{phase}_log.txt'), 'a') as f:
                f.write(str(epoch+1) + '\t' + str(epoch_loss[phase]) + '\n')
        print('Epoch:{}/{} | Training Loss: {train:6f} | Validation Loss: {val:6f}'.format(epoch+1, epochs, **epoch_loss))
          
        torch.save(model.state_dict(), os.path.join(model_dir, f'model-epoch-{epoch+1}.pth'))

        scheduler.step()
    end_time = time.time()
    training_time = end_time - start_time
    print("Total training time in mins is {}".format(training_time//60))
    
if not os.path.exists(logs):
    os.makedirs(logs)
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
train()

In [ ]:

# importing package
import matplotlib.pyplot as plt
import numpy as np

# plot lines
plt.plot(epochs_total, train_accuracy, label = "train_accuracy")
plt.plot(epochs_total, valid_accuracy, label = "validation_accuracy")
plt.legend()
plt.show()

In [ ]:
#Final Model
#Model
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F


class ImgPreprocess(nn.Module):

    def __init__(self, embed_dim):

        super(ImgPreprocess, self).__init__()
        self.model = models.resnet50(pretrained=True)
        in_features = self.model.fc.in_features
        self.model.fc =  nn.Linear(in_features, embed_dim)# remove resnet50 last layer
      

    def forward(self, image):
        with torch.no_grad():
            imagefeatures = self.model(image) # (batch, channel, height, width)
            
        l2_norm = F.normalize(imagefeatures, p=2, dim=1).detach()
        return l2_norm

class QuestionPreprocess(nn.Module):
    def __init__(self, questionvocabularysize, wordembeddingsize, hidden_size, num_hidden, sizeofconcatenatedlayer):
                super(QuestionPreprocess, self).__init__()
                self.word_embedding = nn.Embedding(questionvocabularysize, wordembeddingsize)
                self.tanh = nn.Tanh()
                self.hidden_size=hidden_size
                self.num_hidden=num_hidden
                self.lstm=nn.LSTM(wordembeddingsize,hidden_size,num_hidden,batch_first=True,bidirectional=True)
                self.fc=nn.Linear(hidden_size*2,sizeofconcatenatedlayer)
    def forward(self, question):
                qu_embedding = self.word_embedding(question)  
                qu_embedding = self.tanh(qu_embedding)
                h0=torch.zeros(self.num_hidden*2,qu_embedding.size(0),self.hidden_size).to('cuda')
                c0=torch.zeros(self.num_hidden*2,qu_embedding.size(0),self.hidden_size).to('cuda')
                output,_=self.lstm(qu_embedding,(h0,c0))
                output=self.fc(output[:,-1,:])
                return output
class VizWizModel(nn.Module):

    def __init__(self, sizeoffeatures, questionvocabularysize, answervocabularysize, wordembeddingsize, hidden_size, num_hidden):

        super(VizWizModel, self).__init__()
        self.questionpreprocess = QuestionPreprocess(questionvocabularysize, wordembeddingsize, hidden_size, num_hidden, sizeoffeatures)
        self.imgpreprocess = ImgPreprocess(sizeoffeatures)
        self.tanh = nn.Tanh()
        self.dropout = nn.Dropout(0.5)
        self.layer1 = nn.Linear(sizeoffeatures, answervocabularysize)
        self.layer2 = nn.Linear(answervocabularysize, answervocabularysize)

    def forward(self, image, question):

        imagefeatures = self.imgpreprocess(image)               
        queationfeatures = self.questionpreprocess(question)
        dotproductfeatures = imagefeatures * queationfeatures
        dotproductfeatures = self.dropout(dotproductfeatures)
        dotproductfeatures = self.tanh(dotproductfeatures)
        dotproductfeatures = self.layer1(dotproductfeatures)       
        dotproductfeatures = self.dropout(dotproductfeatures)
        dotproductfeatures = self.tanh(dotproductfeatures)
        logits = self.layer2(dotproductfeatures)                 

        return logits


In [ ]:
preprecessed={}
preprecessed['train']=preprocessingtrain('train')
preprocessed['val']=preprocessing('val')
preprecessed['train-val'] = preprecessed['train'] + preprecessed['val']

In [ ]:
 for key, value in preprecessed.items():
        np.save(os.path.join("./data", f'{key}.npy'), np.array(value))

In [ ]:
import os
import time
import torch
import torch.nn as nn

from torch import optim


train_accuracy=[]
valid_accuracy=[]
epochs_total=[]
datadir = './data'
model_dir = './ckpt'
logs = './log'

workers = 1
batchsize = 100
sizeoffeatures, embeddingsize = 1024, 300
numofhidden, sizeofhiddenlayer = 3, 512

epochs = 12

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device='cuda'
print(device)
def train():
    dataloader = data_loader(input_dir=datadir, batch_size=batchsize, num_worker=workers)
    questionvocabularysize = qvocabsize+1
    answervocabularysize = 3001
    
    model = VizWizModel(sizeoffeatures=sizeoffeatures, questionvocabularysize=questionvocabularysize, answervocabularysize=answervocabularysize,wordembeddingsize=embeddingsize, hidden_size=sizeofhiddenlayer, num_hidden=numofhidden).to(device)
   
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
   
   
    print('>> start training')
    start_time = time.time()
    for epoch in range(epochs):
        metrics=[]
        metrics1=[]
        correct1=0
        length1=0
        correct=0
        length=0
        epoch_loss = {key: 0 for key in ['train', 'val']}
        print(epoch)
        model.train()
        print(dataloader['train'])
        for idx, dataentry in enumerate(dataloader['train']):
            
            image = dataentry['image'].to(device)
            question = dataentry['question'].to(device)
            label = dataentry['answer'].to(device)
            answers=dataentry['answers']
            label1=label.tolist()
            
            # forwardpropogation
            logits = model(image, question)
            
            with torch.no_grad():
                predict = torch.log_softmax(logits, dim=1)
                predict = torch.argmax(predict, dim=1).tolist()
            print("Train set prediction {}".format(Counter(predict)))
            for i in range(len(label1)):
                if(predict[i]==label1[i]):
                    correct+=1
            length=length+batchsize
            for i in range(answers.shape[0]):
                count=0
                for j in range(answers.shape[1]):
                    if(answers[i][j]==predict[i]):
                        count+=1
                metrics.append(min(count//3,1))
            loss = criterion(logits, label)
            epoch_loss['train'] += loss.item()
            # backpropogation
            optimizer.zero_grad()
            loss.backward()
            print(loss.item())
            optimizer.step()
        model.eval()
        correct = correct/length 
        print("Train set Accuracy is {}".format(correct))
        print("Train set Accuracy metric is {}".format(np.mean(metrics)))
        train_accuracy.append(np.mean(metrics))
        
        for idx, dataentry in enumerate(dataloader['val']):
            image = dataentry['image'].to(device=device)
            question = dataentry['question'].to(device=device)
            label = dataentry['answer'].to(device=device)
            answers=dataentry['answers']
            label1=label
            with torch.no_grad():
                logits = model(image, question)
                loss = criterion(logits, label)
            epoch_loss['val'] += loss.item()
            with torch.no_grad():
                predict = torch.log_softmax(logits, dim=1)
                predict = torch.argmax(predict, dim=1).tolist()
            print("Valid set prediction {}".format(Counter(predict)))
            for i in range(len(predict)):
                if(predict[i]==label1[i]):
                    correct1+=1
            length1=length1+batchsize
            for i in range(answers.shape[0]):
                count=0
                for j in range(answers.shape[1]):
                    if(answers[i][j]==predict[i]):
                        count+=1
                metrics1.append(min(count//3,1))
        correct1 = correct1/length1
        print("Validation set Accuracy is {}".format(correct1))
        print("Validation set Accuracy metric is {}".format(np.mean(metrics1)))
        valid_accuracy.append(np.mean(metrics1))
        epochs_total.append(epoch)
        # statistic
        for phase in ['train', 'val']:
            epoch_loss[phase] /= len(dataloader[phase])
            with open(os.path.join(logs, f'{phase}_log.txt'), 'a') as f:
                f.write(str(epoch+1) + '\t' + str(epoch_loss[phase]) + '\n')
        print('Epoch:{}/{} | Training Loss: {train:6f} | Validation Loss: {val:6f}'.format(epoch+1, epochs, **epoch_loss))
          
        torch.save(model.state_dict(), os.path.join(model_dir, f'model-epoch-{epoch+1}.pth'))

        scheduler.step()
    end_time = time.time()
    training_time = end_time - start_time
    print("Total training time in mins is {}".format(training_time//60))
    
if not os.path.exists(logs):
    os.makedirs(logs)
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
train()

In [ ]:
# importing package
import matplotlib.pyplot as plt
import numpy as np

# plot lines
plt.plot(epochs_total, train_accuracy, label = "train_accuracy")
plt.plot(epochs_total, valid_accuracy, label = "validation_accuracy")
plt.legend()
plt.show()

In [ ]:

def preprocessingtest(type1):
        missing_images = []

   
    ## load annotation file and question file
        dataset = json.load(open(root_dir+'/'+type1+'.json'))
        data = [None]*len(dataset)

        for idx,one_data in enumerate(dataset):
#             ans_counter = Counter([x['answer'] for x in one_data['answers']])
#             allanswers = [x['answer'] for x in one_data['answers']]
#             ans = ans_counter.most_common(1)[0][0]
#             a_label = answer2answer_id.get(ans,3000)
#             answers=[]
#             if type1 == 'train' and a_label == -1:
#                 continue
#             for answer in allanswers :
#                 answers.append(answer2answer_id.get(answer,3000))
            
       
            imagename = one_data['image']
            imagepath = "./images_testresized"+"/"+ imagename
            questionsentence=one_data['question']
            questiontokens = nltk.word_tokenize(question)
            questionencoding = sentence_to_vocabid(one_data['question'], vocab)
            if(len(questionencoding)>=37):
                questionencoding=questionencoding[0:37]
            else:
                questionencoding.extend([0]*(37-len(questionencoding)))
            image_info = {'imagename': imagename,
                'imagepath': imagepath,
                'questionsentence': questionsentence,
                'questiontokens': questiontokens
                }
            
            image_info['questionencoding'] = questionencoding
            data[idx]=image_info
        return data

In [ ]:
preprocessed

In [ ]:
preprecessed={}
preprecessed['train']=preprocessingtrain('train')
preprecessed['val']=preprocessingvalidation('val')
preprecessed['test']=preprocessingtest('test')
preprecessed['train-val'] = preprecessed['train'] + preprecessed['val']

In [ ]:
 for key, value in preprecessed.items():
        np.save(os.path.join("./data", f'{key}.npy'), np.array(value))

In [ ]:
ckpt_dir='../input/resizedviz/ckpt/model-epoch-12.pth'

In [ ]:
workers = 1
batchsize = 100
sizeoffeatures, embeddingsize = 1024, 300
numofhidden, sizeofhiddenlayer = 3, 512

In [ ]:
with open('./answer2answer_id.json', 'r') as f:
    answer_data = json.load(f)
    

In [ ]:
def swap(json):
    ret = {}
    for key in json:
        ret[json[key]] = key
  
    return ret

In [ ]:
idtoanswer=swap(answer_data)

In [ ]:
idtoanswer[0]

In [ ]:
questionvocabularysize = qvocabsize+1
answervocabularysize = 3001

In [ ]:
from torchvision import transforms

In [ ]:
device='cuda'

In [ ]:
import csv
predictions=[]
def test(input_dir, data_type, batch_size, num_worker):
    print("hi")
    transform = transforms.Compose([
        transforms.ToTensor(),  # convert to (C,H,W) and [0,1]
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))  # mean=0; std=1
    ])
    VizWiz_dataset = VizWizdataset(input_dir, f'{data_type}.npy', transformimage= transform)
    dataloader = DataLoader(VizWiz_dataset, batch_size=batch_size, shuffle=False, num_workers=num_worker)
   
    model=VizWizModel(sizeoffeatures=sizeoffeatures, questionvocabularysize=questionvocabularysize, answervocabularysize=answervocabularysize,wordembeddingsize=embeddingsize, hidden_size=sizeofhiddenlayer, num_hidden=numofhidden).to(device)
    model.load_state_dict(torch.load(ckpt_dir))
    model.eval()
    results = []
    try:
        for idx, dataentry in enumerate(dataloader):
            image = dataentry['image'].to(device)
            question = dataentry['question'].to(device)
            imagepaths=dataentry['imagepath']
            
            if(idx==1500):
                break
            with torch.no_grad():
               
                logits = model(image, question)
                predict = torch.log_softmax(logits, dim=1)
                predict = torch.argmax(predict, dim=1).tolist()
                
                predict = [[idtoanswer[idx]] for idx in predict]
                predict_final=predict
                
                for i in range(len(predict)):
                     if(predict_final[i][0]=='<unk>'):
                            predict_final[i][0]='unanswerable'
                        
                #imagepath=[[i] for i in imagepaths]
                #pred=[[imagepath[i],predict[i]] for i in range(len(imagepath))]
                predictions.extend(predict_final)
        
    except:
        pass
    file = open('results3.csv', 'w')
    with file:    
            write = csv.writer(file)
            write.writerows(predictions)

    

In [ ]:
test('./data', 'test', batch_size=100, num_worker=1)